In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn
print('sklearn',sklearn.__version__)

2022-05-25 12:47:43.639152
Python 3.8.10
sklearn 1.0.2


In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models

In [23]:
# Data straight from CellProfiler (rows for every patch image file)
BASE_DIR='/home/jrm/Adjeroh/Naved/Process100.class/'
CLASS_DIRS=['0','1','2','3','4','5']
IMG_FN="Process100_Image.csv"
RBC_FN="Process100_MergeRBC.csv"
NUC_FN="Process100_Nucleus.csv"
CEL_FN="Process100_ExpandCells.csv"
TIS_FN="Process100_Tissue.csv"
# Saving our models
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.01/'

In [37]:
# show columns
df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN)
for col in df.columns:
    print(col)

ImageNumber
ObjectNumber
FileName_Tumor
PathName_Tumor
AreaShape_Area
AreaShape_BoundingBoxArea
AreaShape_BoundingBoxMaximum_X
AreaShape_BoundingBoxMaximum_Y
AreaShape_BoundingBoxMinimum_X
AreaShape_BoundingBoxMinimum_Y
AreaShape_Center_X
AreaShape_Center_Y
AreaShape_CentralMoment_0_0
AreaShape_CentralMoment_0_1
AreaShape_CentralMoment_0_2
AreaShape_CentralMoment_0_3
AreaShape_CentralMoment_1_0
AreaShape_CentralMoment_1_1
AreaShape_CentralMoment_1_2
AreaShape_CentralMoment_1_3
AreaShape_CentralMoment_2_0
AreaShape_CentralMoment_2_1
AreaShape_CentralMoment_2_2
AreaShape_CentralMoment_2_3
AreaShape_Compactness
AreaShape_ConvexArea
AreaShape_Eccentricity
AreaShape_EquivalentDiameter
AreaShape_EulerNumber
AreaShape_Extent
AreaShape_FormFactor
AreaShape_HuMoment_0
AreaShape_HuMoment_1
AreaShape_HuMoment_2
AreaShape_HuMoment_3
AreaShape_HuMoment_4
AreaShape_HuMoment_5
AreaShape_HuMoment_6
AreaShape_InertiaTensorEigenvalues_0
AreaShape_InertiaTensorEigenvalues_1
AreaShape_InertiaTensor_0_0
Ar

In [63]:
df = None
GOOD_COLS=['ImageNumber','AreaShape_Area','AreaShape_MeanRadius','Neighbors_NumberOfNeighbors_Expanded']
def nuc_prefix(col_name):
    return 'Nuc_'+col_name
def load_one_class(class_index):
    class_name=CLASS_DIRS[class_index]+'/'
    object_df = pd.read_csv(BASE_DIR+class_name+NUC_FN)
    print("Load %d nuclei." % len(object_df))
    object_df = object_df[GOOD_COLS]
    object_df.rename(nuc_prefix,axis='columns',inplace=True)
    patch_df = object_df.groupby(['Nuc_ImageNumber']).describe()
    print("Load %d patches having nuclei." % len(patch_df))
    return patch_df
df = load_one_class(0)

Load 1826 nuclei.
Load 100 patches having nuclei.


In [64]:
df

Nuc_AreaShape_Area                                         \
                             count        mean         std    min     25%   
Nuc_ImageNumber                                                             
1                              1.0  313.000000         NaN  313.0  313.00   
2                              3.0  251.000000   79.680612  178.0  208.50   
3                              5.0  302.000000  162.260593  182.0  212.00   
4                             29.0  314.137931  125.210030  177.0  233.00   
5                             32.0  355.937500  160.779339  180.0  243.00   
...                            ...         ...         ...    ...     ...   
96                             2.0  306.500000   47.376154  273.0  289.75   
97                            32.0  394.750000  171.982745  189.0  270.50   
98                            23.0  318.565217  120.095729  183.0  244.50   
99                             2.0  304.500000  116.672619  222.0  263.25   
100                           16.0  423.312500  221.529748  221.0  263.00   

                                      Nuc_AreaShape_MeanRadius            ...  \
                   50%     75%    max                    count      mean  ...   
Nuc_ImageNumber                                                           ...   
1                313.0  313.00  313.0                      1.0  3.387379  ...   
2                239.0  287.50  336.0                      3.0  2.472544  ...   
3                260.0  271.00  585.0                      5.0  3.278895  ...   
4                262.0  340.00  635.0                     29.0  3.072118  ...   
5                331.0  375.25  832.0                     32.0  3.188709  ...   
...                ...     ...    ...                      ...       ...  ...   
96               306.5  323.25  340.0                      2.0  3.069114  ...   
97               352.5  480.75  812.0                     32.0  3.372149  ...   
98               285.0  373.00  633.0                     23.0  3.056807  ...   
99               304.5  345.75  387.0                      2.0  1.997744  ...   
100              304.0  552.75  997.0                     16.0  3.471893  ...   

                                    Nuc_Neighbors_NumberOfNeighbors_Expanded  \
                      75%       max                                    count   
Nuc_ImageNumber                                                                
1                3.387379  3.387379                                      1.0   
2                2.754673  3.177413                                      3.0   
3                3.339997  4.695143                                      5.0   
4                3.247382  3.940398                                     29.0   
5                3.514243  4.589244                                     32.0   
...                   ...       ...                                      ...   
96               3.317004  3.564894                                      2.0   
97               3.808400  4.650069                                     32.0   
98               3.390141  4.031012                                     23.0   
99               2.052328  2.106912                                      2.0   
100              4.100576  4.582329                                     16.0   

                                                               
                     mean       std  min  25%  50%   75%  max  
Nuc_ImageNumber                                                
1                0.000000       NaN  0.0  0.0  0.0  0.00  0.0  
2                2.000000  0.000000  2.0  2.0  2.0  2.00  2.0  
3                2.400000  0.894427  1.0  2.0  3.0  3.00  3.0  
4                4.689655  1.649750  2.0  4.0  4.0  5.00  8.0  
5                4.687500  1.635050  2.0  3.0  4.5  5.25  8.0  
...                   ...       ...  ...  ...  ...   ...  ...  
96               1.000000  0.000000  1.0  1.0  1.0  1.00  1.0  
97               4.812500  1.554131  2.0 